In [2]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
import random

delphesDir = os.path.abspath("../../MG5/Delphes")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT
import xml.etree.ElementTree as ET


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

Welcome to JupyROOT 6.24/06


True

In [3]:
inputFile = '../../processFolders/sm_ttmatch_lo/Events/run_01/tag_1_delphes_events.root'
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
f.Close()

In [18]:
etamax=2.0
pTmin=355.0

atlas_bins = [355.0,381.0,420.0,478.0,549.0,633.0,720.0,836.0,2000.0]

f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
ntotal = 0
totalweightPB = 0.0
pTT = []
weights = []
for ievt in range(nevts):    
            
    ntotal += 1
    tree.GetEntry(ievt)
    weightPB = tree.Weight.At(1).Weight
    weightPB = weightPB
    totalweightPB += weightPB

    genParticles = tree.Particle
    # Filter tops:
    tops = {}
    # topList = {}
    # Filter Ws
    # wList = {}
    for iptc in range(genParticles.GetEntries()):
        ptc = genParticles.At(iptc)
        if abs(ptc.PID) == 6:
            # topList[iptc] = ptc # Store all tops generated during event evolution
            tops[ptc.PID] = ptc # Store only the last top/anti-top
        # elif abs(ptc.PID) == 24:
            # wList[iptc] = ptc

    # Get top decays:
    ws = {}
    bjets = {}
    for itop,top in tops.items():
        for idaugther in range(top.D1,top.D2+1):
            ptc = genParticles.At(idaugther)
            if abs(ptc.PID) == 24:
                ws[ptc.PID] = ptc
            elif abs(ptc.PID) == 5:
                bjets[ptc.PID] = ptc
    
    # Get W decays:
    wDecays = {}
    for iw,w in ws.items():
        wDecays[w.PID] = {}
        d1 = w.D1
        d2 = w.D2+1
        daughter1 = genParticles.At(d1)
        while daughter1.PID == w.PID:
            d1 = daughter1.D1
            d2 = daughter1.D2+1
            daughter1 = genParticles.At(d1)
        for idaugther in range(d1,d2):
            ptc = genParticles.At(idaugther)
            wDecays[w.PID][ptc.PID] = ptc

    break
    # Randomly select one top:
    itop = random.choice([6,-6])
    itop = 6
    pT = tops[itop].PT
    eta = np.abs(tops[itop].Eta)
    if pT < pTmin:
        continue
    if eta > etamax:
        continue
    pTT.append(pT)
    weights.append(weightPB)
    
weights = np.array(weights)
pTtHist,_ = np.histogram(pTT,weights=weights,bins=atlas_bins)
pTtHistError,_ = np.histogram(pTT,weights=weights**2,bins=atlas_bins)
pTtHistError = np.sqrt(pTtHistError)

data = np.array(list(zip(atlas_bins[:-1],atlas_bins[1:],pTtHist,pTtHistError)))
    

In [19]:
tops

{6: <cppyy.gbl.GenParticle object at 0x16906b50>,
 -6: <cppyy.gbl.GenParticle object at 0x16906bc0>}

In [20]:
ws

{24: <cppyy.gbl.GenParticle object at 0x167c4490>,
 -24: <cppyy.gbl.GenParticle object at 0x167c46c0>}

In [21]:
wDecays

{24: {-1: <cppyy.gbl.GenParticle object at 0x167c48f0>,
  2: <cppyy.gbl.GenParticle object at 0x167c4960>},
 -24: {3: <cppyy.gbl.GenParticle object at 0x167c5c30>,
  -4: <cppyy.gbl.GenParticle object at 0x167c5ca0>}}

In [33]:
kfac = 832./totalweightPB
BR = 0.6741*(0.1071+0.1063)*2
print('xsec (pb) = %1.3e' %totalweightPB)
print('fiducial xsec (pb) = %1.3e' %sum(weights))
print('fiducial xsec*BR (pb) = %1.3e' %(sum(weights)*BR))
print('fiducial xsec*BR*kfac (pb) = %1.3e' %(sum(weights)*BR*kfac))


xsec (pb) = 4.653e+02
fiducial xsec (pb) = 8.387e+00
fiducial xsec*BR (pb) = 2.413e+00
fiducial xsec*BR*kfac (pb) = 4.315e+00
